In [363]:
import math

In [364]:
#This outputs single poisson probability
def get_poisson_prob(parameter, n):
    return (parameter**n / math.factorial(n)) * math.exp(-parameter)

#This outputs joint poisson probability
def get_joint_prob(request, back, parking):
    if parking == 'a':
        return get_poisson_prob(3,request)*get_poisson_prob(3,back)
    return get_poisson_prob(4,request)*get_poisson_prob(2,back)

#Get the action space of given state
def get_legal_action(state_a, state_b):
    result = []
    for action in range(-20,21):
        if -state_b <= action <= state_a and action+state_b <= 20 and state_a-action <= 20:
            result.append(action)
    return result

#Help determine the possible combos of request and return in state transition
def request_return_boundaries(diff, state, new):
    if diff > 0:
        return 0, diff, state
    return -diff, 0, new

In [365]:
#Single instance transition state function
def state_transition_prob(new_a, new_b, state_a, state_b, action):
    global final_prob_a, final_prob_b
    state_a -= action
    state_b += action
    request_start_a, back_start_a, gap_a = request_return_boundaries(new_a - state_a, state_a, new_a)
    request_start_b, back_start_b, gap_b = request_return_boundaries(new_b - state_b, state_b, new_b)
    prob_sum_a = sum(get_joint_prob(request_start_a+i, back_start_a+i, 'a') for i in range(gap_a))
    prob_sum_b = sum(get_joint_prob(request_start_b+i, back_start_b+i, 'b') for i in range(gap_b))
    if new_a == 20:
        final_prob_a = (1-sum(get_poisson_prob(3,i) for i in range(request_start_a+gap_a))) * (1-sum(get_poisson_prob(3,j) for j in range(20)))
    elif new_a <= 20:
        final_prob_a = (1-sum(get_poisson_prob(3,i) for i in range(request_start_a+gap_a))) * get_poisson_prob(3,back_start_a+gap_a)
    if new_b == 20:
        final_prob_b = (1-sum(get_poisson_prob(4,i) for i in range(request_start_b+gap_b))) * (1-sum(get_poisson_prob(2,j) for j in range(20)))
    elif new_b <= 20:
        final_prob_b = (1-sum(get_poisson_prob(4,i) for i in range(request_start_b+gap_b))) * get_poisson_prob(2,back_start_b+gap_b)
    prob_sum_a += final_prob_a
    prob_sum_b += final_prob_b
    return prob_sum_a*prob_sum_b


#Print all the cases for old state of (3,3)
for a in get_legal_action(3,3):
    print('when action = {} the transition probability is {}'.format(a, state_transition_prob(3,3,3,3,a)))

#Get the complete dictionary of state transition probabilities
def all_transition_prob():
    dict = {}
    for new_a in range(21):
        for new_b in range(21):
            for state_a in range(21):
                for state_b in range(21):
                    for action in get_legal_action(state_a,state_b):
                        dict['{},{},{},{},{}'.format(new_a,new_b,state_a,state_b,action)] = state_transition_prob(new_a,new_b,state_a,state_b,action)
    return dict

complete_transition_prob = all_transition_prob()


when action = -3 the transition probability is 0.014010000047189993
when action = -2 the transition probability is 0.02251703576649734
when action = -1 the transition probability is 0.03210495745102386
when action = 0 the transition probability is 0.04073020718020566
when action = 1 the transition probability is 0.0450966730040062
when action = 2 the transition probability is 0.04314269768441976
when action = 3 the transition probability is 0.03593556512575768


In [366]:
#Single instance reward function
def compute_reward(state_a, state_b, action):
    state_a -= action
    state_b += action
    lent_out_a, lent_out_b, sum_prob_a, sum_prob_b = 0, 0, 0, 0
    for i in range(state_a+1):
        lent_out_a += i*get_poisson_prob(3,i)
        sum_prob_a += get_poisson_prob(3,i)
    for j in range(state_b+1):
        lent_out_b += j*get_poisson_prob(4,j)
        sum_prob_b += get_poisson_prob(4,j)
    return 10*(lent_out_a + lent_out_b + state_a*(1-sum_prob_a) + state_b*(1-sum_prob_b)) - 2*abs(action)

#Print all the cases for state of (3,3)
for a in get_legal_action(3,3):
    print('when action = {} the reward is {}'.format(a, compute_reward(3,3,a)))

#Get the complete dictionary for all rewards
def all_reward():
    dict = {}
    for state_a in range(21):
        for state_b in range(21):
            for action in get_legal_action(state_a,state_b):
                dict['{},{},{}'.format(state_a,state_b,action)] = compute_reward(state_a,state_b,action)
    return dict

complete_reward = all_reward()


when action = -3 the reward is 23.492973857591373
when action = -2 the reward is 34.47063804839099
when action = -1 the reward is 43.70748854919201
when action = 0 the reward is 49.79877438147888
when action = 1 the reward is 47.695973989080215
when action = 2 the reward is 41.39908737199604
when action = 3 the reward is 32.04565418537064


In [367]:
#Initialize state value dictionary
complete_state_value = {}
for i in range(21):
    for j in range(21):
        complete_state_value[f'{i},{j}'] = 0

discount_rate = 0.9
threshold = 0.001

#Performs value iteration and also remembers optimal action along the search
#The greedy policy function is combined here since there is little extra cost
#Outputs both the optimal value and the optimal action
def value_iteration(transition_dict, reward_dict, value_dict):
    while True:
        action_dict = {}
        theta = 0
        for state_combo in value_dict.keys():
            state_combo = state_combo.split(',')
            state_a, state_b = int(state_combo[0]), int(state_combo[1])
            #different_action_results = []
            best_action, max_value = 0,0
            for action in get_legal_action(state_a,state_b):
                sum_of_s_prime = 0
                for new_a in range(21):
                    for new_b in range(21):
                        sum_of_s_prime += transition_dict[f'{new_a},{new_b},{state_a},{state_b},{action}'] * value_dict[f'{new_a},{new_b}']
                candidate = reward_dict[f'{state_a},{state_b},{action}'] + discount_rate*sum_of_s_prime
                #different_action_results.append(candidate)
                if candidate > max_value:
                    max_value = candidate
                    best_action = action
            action_dict[f'{state_a},{state_b}'] = best_action
            old_value = value_dict[f'{state_a},{state_b}']
            #value_dict[f'{state_a},{state_b}'] = max(different_action_results)
            value_dict[f'{state_a},{state_b}'] = max_value
            theta = max(theta, abs(value_dict[f'{state_a},{state_b}'] - old_value))
        if theta < threshold:
            return value_dict, action_dict


optimal_state_value, optimal_action = value_iteration(complete_transition_prob, complete_reward, complete_state_value)

In [368]:
print(optimal_state_value['3,3'])

478.9495217329611


In [369]:
#Finding the optimal action
#the same stuff is already implemented in value iteration above
#this displays how to do greedy policy separately
def determine_greedy_policy(transition_dict, reward_dict, value_dict):
    policy_dict = {}
    for state_combo in value_dict.keys():
        state_combo = state_combo.split(',')
        state_a, state_b = int(state_combo[0]), int(state_combo[1])
        best_result, best_action = 0,0
        for action in get_legal_action(state_a,state_b):
            sum_of_s_prime = 0
            for new_a in range(21):
                for new_b in range(21):
                    sum_of_s_prime += transition_dict[f'{new_a},{new_b},{state_a},{state_b},{action}'] * value_dict[f'{new_a},{new_b}']
            candidate = reward_dict[f'{state_a},{state_b},{action}'] + sum_of_s_prime*discount_rate
            if candidate > best_result:
                best_result = candidate
                best_action = min(action,5)
        policy_dict[f'{state_a},{state_b}'] = best_action
    return policy_dict

greedy_policy = determine_greedy_policy(complete_transition_prob, complete_reward, optimal_state_value)

In [370]:
for i in range(21):
    print(f'The action to take at state ({i},3) is  ', greedy_policy[f'{i},3'])

The action to take at state (0,3) is   0
The action to take at state (1,3) is   0
The action to take at state (2,3) is   0
The action to take at state (3,3) is   0
The action to take at state (4,3) is   0
The action to take at state (5,3) is   0
The action to take at state (6,3) is   1
The action to take at state (7,3) is   1
The action to take at state (8,3) is   2
The action to take at state (9,3) is   2
The action to take at state (10,3) is   3
The action to take at state (11,3) is   3
The action to take at state (12,3) is   3
The action to take at state (13,3) is   3
The action to take at state (14,3) is   4
The action to take at state (15,3) is   4
The action to take at state (16,3) is   5
The action to take at state (17,3) is   5
The action to take at state (18,3) is   5
The action to take at state (19,3) is   5
The action to take at state (20,3) is   5
